In [11]:
import argparse
import os
import numpy as np
from preprocessing import parse_annotation
from frontend import YOLO
import json
#from keras.utils.training_utils import multi_gpu_model
from keras.utils import multi_gpu_model

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"


In [12]:
config_path = "test/config.json"
with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

os.environ["CUDA_VISIBLE_DEVICES"] = config['env']['gpu']
gpus = max(1, len(config['env']['gpu'].split(",")))

In [ ]:

###############################
#   Parse the annotations 
###############################

# parse annotations of the training set
train_imgs, train_labels = parse_annotation(config['train']['train_annot_folder'], 
                                            config['train']['train_image_folder'], 
                                            config['model']['labels'], ".jpg")

# parse annotations of the validation set, if any, otherwise split the training set
if os.path.exists(config['valid']['valid_annot_folder']):
    valid_imgs, valid_labels = parse_annotation(config['valid']['valid_annot_folder'], 
                                                config['valid']['valid_image_folder'], 
                                                config['model']['labels'], ".jpg")
else:
    train_valid_split = int(0.8*len(train_imgs))
    np.random.shuffle(train_imgs)
    
    valid_imgs = train_imgs[train_valid_split:]
    train_imgs = train_imgs[:train_valid_split]


overlap_labels = set(config['model']['labels']).intersection(set(train_labels.keys()))

print ('Seen labels:{}'.format(train_labels))
print ('Given labels:{}'.format(config['model']['labels']))
print ('Overlap labels:{}'.format(overlap_labels))    

if len(overlap_labels) < len(config['model']['labels']):
    print ('Some labels have no images! Please revise the list of labels in the config.json file!')
else:
    ###############################
    #   Construct the model 
    ###############################

    yolo = YOLO(architecture        = config['model']['architecture'],
                input_size          = config['model']['input_size'], 
                labels              = config['model']['labels'], 
                max_box_per_image   = config['model']['max_box_per_image'],
                anchors             = config['model']['anchors'],
                gpus                = gpus)

    ###############################
    #   Load the pretrained weights (if any) 
    ############################### 
    
    if os.path.exists(config['train']['pretrained_weights']):
       print ("Loading pre-trained weights in {}".format(config['train']['pretrained_weights']))
       yolo.load_weights(config['train']['pretrained_weights'])

    ###############################
    #   Start the training process 
    ###############################
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
    yolo.train(train_imgs         = train_imgs,
              valid_imgs         = valid_imgs,
              train_times        = config['train']['train_times'],
              valid_times        = config['valid']['valid_times'],
              nb_epoch           = config['train']['nb_epoch'], 
              learning_rate      = config['train']['learning_rate'], 
              batch_size         = config['train']['batch_size'],
              warmup_bs          = config['train']['warmup_batches'],
              object_scale       = config['train']['object_scale'],
              no_object_scale    = config['train']['no_object_scale'],
              coord_scale        = config['train']['coord_scale'],
              class_scale        = config['train']['class_scale'],
              saved_weights_name = config['train']['saved_weights_name'],
              debug              = config['train']['debug'])


Seen labels:{'RBC': 3854}
Given labels:['RBC']
Overlap labels:{'RBC'}
(13, 13)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
model_4 (Model)                 (None, 13, 13, 1024) 50547936    input_4[0][0]                    
__________________________________________________________________________________________________
conv_23 (Conv2D)                (None, 13, 13, 30)   30750       model_4[1][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 13, 13, 5, 6) 0           conv_23[0][0]                    
______________________________